In [ ]:
import siibra
from nilearn import plotting, image
import requests
import matplotlib.pyplot as plt
%matplotlib notebook

In [ ]:
assert(siibra.__version__=='0.2a5')
atlas = siibra.atlases['human']

# Accessing locations of brain regions in BigBrain space

## Finding regions which are mapped in BigBrain

Some regions are mapped in BigBrain space. 

In [ ]:
bigbrain = atlas.get_template('bigbrain').fetch()
cytomaps = atlas.get_map(space="bigbrain").fetch()
plotting.view_img(cytomaps,bg_img=bigbrain,cmap='jet',symmetric_cmap=False)

However, in the MNI space, we have of course access to many more existing maps, also in the form of probabilistic maps.

In [ ]:
pmaps = atlas.get_map(space="mni152",maptype="continuous")
len(pmaps)

Let's just fetch one of them.

In [ ]:
img = pmaps.fetch(mapindex=94)
plotting.plot_stat_map(img)

Siibra keeps track of indices and regions, so we can easily determin which region this is:

In [ ]:
pmaps.decode_label(mapindex=94)

## Projecting probabilistic maps from MNI space to BigBrain

If a region is not mapped in the target space, `siibra` will try to warp bounding boxes across spaces, if needed. If desired, we can compute the bounding box by thresholding probability maps. This is preferred for more control of the mask creation.

In [ ]:
region = atlas.get_region("hoc5 left")
voi = region.get_bounding_box("bigbrain", threshold_continuous=0.8)

### Retrieve BigBrain image data and maps for the region of interest

We can apply the bounding box defined above for fetching image data from BigBrain space.

In [ ]:
# Get access to the template definition of BigBrain
bigbrain = atlas.get_template("bigbrain")

# Fetch actual BigBrain data, but only inside the volume of interest. 
# Try full resolution (siibra complains if this leads to prohibitively large data)
siibra.set_feasible_download_size(0.1)
img = bigbrain.fetch(voi=voi, resolution_mm=-1)

We can now reuse the same bounding box to access the layer segmentations of Konrad Wagstyl.

In [ ]:
layermap = atlas.get_map(parcellation='layers', space='bigbrain')
labels = layermap.fetch(voi=voi, resolution_mm=-1)

The returned data is a spatial image object, defined in the physical coordinate system. We can therefore plot image data and maps, despite possibly different image resolution. 

In [ ]:
plotting.view_img(labels, bg_img=img, opacity=.1, symmetric_cmap=False)

Since we work in physical coordinates, we can fetch the whole brain template at a lower resolution and simply plot the full-resolution volume of interest on top of it.

In [ ]:
# where in the whole brain is this? Show the chunk on top of the lower-resolution whole brain template.
template = bigbrain.fetch(resolution_mm=-1)
plotting.plot_roi(labels, bg_img=template, annotate=True, draw_cross=True)#, cmap='gray' )

### Sample gray values around specific regions

We can also extract peaks of high probability from probability maps in MNI space, warp them to BigBrain space, and extract their surrounding in the BigBrain space.

In [ ]:
# Peaks of v1 left in MNI space
v1l = atlas.get_region('v1 left')
peaks, pmap = v1l.find_peaks("mni152")
view = plotting.plot_stat_map(pmap, cmap='viridis')
view.add_markers([tuple(p) for p in peaks])

In [ ]:
# greap a cube at the corresponding position in BigBrain space
p = peaks[2].warp("bigbrain")
pvoi = p.get_enclosing_cube(width_mm=3)
chunk = atlas.get_template('bigbrain').fetch(voi=pvoi, resolution_mm=-1)

template = atlas.get_template('bigbrain').fetch()
view = plotting.plot_anat(template, cut_coords=tuple(p))
view.add_markers([tuple(p)])
view.add_overlay(chunk, cmap='gray')

In [ ]:
# get the layer masks for the extracted cube
layers = atlas.get_map(parcellation='layers',space='bigbrain').fetch(voi=pvoi,resolution_mm=-1)
plotting.view_img(layers, bg_img=chunk, opacity=0.1, symmetric_cmap=False)

In [ ]:
import numpy as np

# access the data array of the chunk
A = chunk.get_fdata()

# the maps might have a voxel space - resample the maps
L = image.resample_to_img(layers,chunk).get_fdata()

# extract histograms per layer
binwidth = 20
bins = range(0,255,binwidth)
histograms = {}
for layer in range(1,int(L.max())):
    print("layer",layer)
    vals, edges = np.histogram(A[(L==layer) & (A<255)].ravel(), bins, normed=True)
    histograms[layer] = vals

In [ ]:
# plot the histograms
fig, ax = plt.subplots()
width=binwidth/8
x = np.array(bins)[1:]
for layer,values in histograms.items():
    bars1 = ax.bar(x+(layer-4)*width, values, width, label=f'Layer {layer}')
plt.xticks(x)
plt.legend([f'Layer {i+1}' for i in range(7)])
plt.grid(True)
plt.show()